In [1]:
import pandas as pd
import os
root_dir = os.getcwd()
from data_loading import load_ftn, load_stadium_data, load_weather_data
from data_cleaning import feature_engineering


In [2]:
df_games=pd.read_csv('data/games.csv')
    
df_games=pd.merge(df_games, load_stadium_data(),left_on='gameId', right_on='old_game_id', how='left')
df_games=pd.merge(df_games, load_weather_data(),on='gameId', how='left')

2022 done.
Downcasting floats.


In [3]:
df_plays = feature_engineering(pd.read_csv('data/plays.csv'))

In [4]:
ftn_merged = load_ftn()

2022 done.
Downcasting floats.
Downcasting floats.


/Users/ccollins/Projects/NFL_BDB2025/data_loading.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_ids['play_id'] = pbp_ids['play_id'].astype(int)
/Users/ccollins/Projects/NFL_BDB2025/data_loading.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_ids['game_id'] = pbp_ids['game_id'].astype(str)


In [5]:
merged_id_df = pd.read_csv('data/merged_ids.csv')

In [6]:
xp_fname = os.path.join(root_dir, "data/team_xpass_ratios.csv")
pr_fname = os.path.join(root_dir, "data/team_pass_rates.csv")
cf_fname = os.path.join(root_dir, "data/coverages_faced.csv")
cu_fname = os.path.join(root_dir, 'data/coverages_used.csv')

In [7]:
xp_df = pd.read_csv(xp_fname).drop(columns='Unnamed: 0')
pr_df = pd.read_csv(pr_fname).drop(columns='Unnamed: 0')

In [8]:
xp_df.head(3)

,gameId,playId,off_xpass,def_xpass
0,2022102302,2655,0.779215,0.723996
1,2022091809,3698,0.685163,0.425815
2,2022103004,3146,0.628021,0.757653


In [9]:
xp_df.isna().sum()

gameId          0
playId          0
off_xpass    2620
def_xpass    2594
dtype: int64

In [10]:
merged_base = merged_id_df.merge(ftn_merged,how='left',on=['gameId','playId'])
merged_base = merged_base.merge(pr_df,how='left',on=['gameId','playId'])

In [11]:
%% capture
ftn_merged=load_ftn()

2022 done.
Downcasting floats.
Downcasting floats.


/Users/ccollins/Projects/NFL_BDB2025/data_loading.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_ids['play_id'] = pbp_ids['play_id'].astype(int)
/Users/ccollins/Projects/NFL_BDB2025/data_loading.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_ids['game_id'] = pbp_ids['game_id'].astype(str)


In [12]:
#ftn_merged

In [13]:
merged_base = merged_id_df.merge(ftn_merged,how='left',on=['gameId','playId'])


In [14]:

merged_base = merged_base.merge(df_games[['gameId','week']].drop_duplicates(),how='left',on=['gameId'])

In [15]:
merged_base = merged_base.merge(df_plays[['gameId','playId',
                                          'possessionTeam','defensiveTeam']].drop_duplicates(),
                                how='left',on=['gameId','playId'])

In [16]:
merged_base.head(3)

,gameId,playId,n_offense_backfield,n_defense_box,is_no_huddle,is_motion,week,possessionTeam,defensiveTeam
0,2022090800,56,1.0,6.0,False,False,1,BUF,LA
1,2022090800,80,1.0,6.0,True,True,1,BUF,LA
2,2022090800,101,1.0,7.0,False,True,1,BUF,LA


In [17]:
mb_w1 = merged_base[merged_base['week'] == 1]

In [18]:
mb_w2_on = merged_base[merged_base['week'] > 1]

In [19]:
w1_ids = mb_w1[['gameId','playId','week','possessionTeam','defensiveTeam']]

In [20]:
w2_on_ids = mb_w2_on[['gameId','playId','week','possessionTeam','defensiveTeam']]

In [21]:
w1_ids

,gameId,playId,week,possessionTeam,defensiveTeam
0,2022090800,56,1,BUF,LA
1,2022090800,80,1,BUF,LA
2,2022090800,101,1,BUF,LA
3,2022090800,122,1,BUF,LA
4,2022090800,167,1,BUF,LA
...,...,...,...,...,...
1945,2022091200,3723,1,DEN,SEA
1946,2022091200,3747,1,DEN,SEA
1947,2022091200,3795,1,DEN,SEA
1948,2022091200,3826,1,DEN,SEA


# Work on data_21

Note to do ftn data later maybe, as to not mess up id's (probably has no effect on id's )

In [22]:
cov_21 = pd.read_csv('data_21/cov_21.csv')
cov_21.head(2)

,possessionTeam,Man_def,Zone_def,cover_0_def,cover_1_def,cover_2_def,cover_3_def,Quarters_def,cover_6_def,Other_def,Man_off,Zone_off,cover_0_off,cover_1_off,cover_2_off,cover_3_off,Quarters_off,cover_6_off,Other_off
0,ARI,9.529412,23.647059,3.117647,5.352941,4.000000,12.588235,4.117647,2.882353,1.117647,7.411765,28.352941,1.823529,5.588235,4.588235,14.000000,5.882353,3.705882,0.176471
1,ATL,11.117647,22.000000,1.647059,9.000000,12.294118,8.117647,0.705882,0.647059,0.705882,8.058824,24.705882,1.235294,6.294118,5.588235,11.294118,5.882353,1.823529,0.647059


In [23]:
team_pr_21 = pd.read_csv('data_21/team_pr_21.csv')
team_pr_21.head(2)

,possessionTeam,pass_rate_def,pass_rate_off
0,ARI,0.573372,0.573464
1,ATL,0.548059,0.603575


# Calculate team pass rates

Note: use team pass rate from last year as xpass for week 1 (due to 2021 data incompleteness)

In [24]:
pr_df.head(1)

,gameId,playId,pass_rate_off,pass_rate_def
0,2022102302,2655,0.654054,0.648794


In [25]:
w1_ids.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam
0,2022090800,56,1,BUF,LA


In [78]:
w1_pr = w1_ids.merge(team_pr_21.drop(columns=['pass_rate_off']).rename(columns={'possessionTeam':'defensiveTeam'}),on='defensiveTeam')

In [80]:
w1_pr.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def
0,2022090800,56,1,BUF,LA,0.615627


In [82]:
w1_pr = w1_pr.merge(team_pr_21.drop(columns=['pass_rate_def']),on='possessionTeam')

In [83]:
w1_pr.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off
0,2022090800,56,1,BUF,LA,0.615627,0.619126


We bake in xpass here because it's just going to replicate pass_rate_off/def for week 1

In [84]:
w1_pr['off_xpass'] = w1_pr['pass_rate_off'].copy()
w1_pr['def_xpass'] = w1_pr['pass_rate_def'].copy()

In [85]:
w1_pr.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627


### Calculate coverage data

Here we just use defensive rates for simplicity, can revisit later

In [86]:
cov_21.head(1)

,possessionTeam,Man_def,Zone_def,cover_0_def,cover_1_def,cover_2_def,cover_3_def,Quarters_def,cover_6_def,Other_def,Man_off,Zone_off,cover_0_off,cover_1_off,cover_2_off,cover_3_off,Quarters_off,cover_6_off,Other_off
0,ARI,9.529412,23.647059,3.117647,5.352941,4.0,12.588235,4.117647,2.882353,1.117647,7.411765,28.352941,1.823529,5.588235,4.588235,14.0,5.882353,3.705882,0.176471


In [87]:
cov_def = cov_21[[x for x in cov_21.columns if '_off' not in x]].rename(columns={'possessionTeam':'defensiveTeam'})

In [88]:
cov_def = cov_def.rename(columns={'cover_2_def':'Cover-2_def','cover_0_def':'Cover-0_def'})

In [89]:
cov_def.head(1)

,defensiveTeam,Man_def,Zone_def,Cover-0_def,cover_1_def,Cover-2_def,cover_3_def,Quarters_def,cover_6_def,Other_def
0,ARI,9.529412,23.647059,3.117647,5.352941,4.0,12.588235,4.117647,2.882353,1.117647


In [105]:
w1_merged = w1_pr.merge(cov_def,how='left',on='defensiveTeam')

In [106]:
w1_merged.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass,Man_def,Zone_def,Cover-0_def,cover_1_def,Cover-2_def,cover_3_def,Quarters_def,cover_6_def,Other_def
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627,7.117647,37.117647,0.705882,6.352941,0.647059,20.823529,7.647059,7.705882,0.352941


don't need to roll imputed values into rolling pass ratio means b/c week 1 already being considered down the line

also consider column ordering: maybe put the pass rate, coverage data last, concat independently, then merge both in

# TODO: train/test flag (i.e., train does w1/w2 bs)

we dont need to modify test processing?

# TODO: maybe weigh a ratio of coverage used 75%, coverage faced 25%

Unsure if can do here or have to go back to coverage processing; think safest to go all the way back and do ratio, depending

Idea is, likely matters far more what teams are employing, not what teams tend to run against offenses

Before we even try that, just do 100% defensive, then slot in the ratio later

In [107]:
w2_on_ids.head()

,gameId,playId,week,possessionTeam,defensiveTeam
1950,2022091500,55,2,KC,LAC
1951,2022091500,76,2,KC,LAC
1952,2022091500,97,2,KC,LAC
1953,2022091500,118,2,KC,LAC
1954,2022091500,142,2,KC,LAC


In [207]:
cu_df = pd.read_csv('data/coverages_used.csv')

In [208]:
cu_df.head(1)

,week,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,defensiveTeam
0,2,21.0,2.0,21.0,7.0,4.0,3.0,26.0,4.0,34.0,ARI


In [209]:
cu_df['week'] = cu_df['week'].astype(int)

In [211]:
w2_on_ids.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam
1950,2022091500,55,2,KC,LAC


In [213]:
cu_w2_on = w2_on_ids.merge(cu_df,how='left',left_on=['defensiveTeam','week'],right_on=['defensiveTeam','week'])

# TODO: figure out what the hell your logic is for coverages utilized/faced

CU is defense, and CF is offense, which is fine. 

In [133]:
xp_df = pd.read_csv(xp_fname).drop(columns='Unnamed: 0')
pr_df = pd.read_csv(pr_fname).drop(columns='Unnamed: 0')

In [134]:
pr_df.head(1)

,gameId,playId,pass_rate_off,pass_rate_def
0,2022102302,2655,0.654054,0.648794


In [219]:
pr_df.isna().sum()

gameId              0
playId              0
pass_rate_off    2670
pass_rate_def    2665
dtype: int64

In [221]:
cu_w2_on.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0


In [227]:
df_w2_on = cu_w2_on.merge(pr_df,how='left',on=['gameId','playId'])

In [228]:
df_w2_on.head(2)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.625,0.763636
1,2022091500,76,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.625,0.763636


# TODO: bye nonsense for pass rate

In [229]:
df_w2_on[df_w2_on.pass_rate_off.isna()]

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,pass_rate_off,pass_rate_def
3289,2022092511,55,3,SEA,ATL,38.0,1.0,21.0,12.0,24.0,4.0,26.0,14.0,75.0,NaN,0.649573
3290,2022092511,76,3,SEA,ATL,38.0,1.0,21.0,12.0,24.0,4.0,26.0,14.0,75.0,NaN,0.649573
3291,2022092511,100,3,SEA,ATL,38.0,1.0,21.0,12.0,24.0,4.0,26.0,14.0,75.0,NaN,0.649573
3292,2022092511,121,3,SEA,ATL,38.0,1.0,21.0,12.0,24.0,4.0,26.0,14.0,75.0,NaN,0.649573
3293,2022092511,142,3,SEA,ATL,38.0,1.0,21.0,12.0,24.0,4.0,26.0,14.0,75.0,NaN,0.649573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12476,2022103012,3403,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,NaN,0.513021
12477,2022103012,3435,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,NaN,0.513021
12478,2022103012,3477,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,NaN,0.513021
12479,2022103012,3498,8,BUF,GB,141.0,32.0,82.0,66.0,15.0,19.0,106.0,19.0,254.0,NaN,0.513021


In [ ]:
df_w2_on.isna()

In [226]:
df_w2_on.isna().sum()

gameId              0
playId              0
week                0
possessionTeam      0
defensiveTeam       0
cover_3_def         0
cover_6_def         0
cover_1_def         0
Quarters_def        0
Cover-2_def         0
Cover-0_def         0
Man_def             0
Other_def           0
Zone_def            0
pass_rate_off     562
pass_rate_def     606
dtype: int64

In [ ]:
df_w2_on = df_w2_on.merge(xp_df,how='left',on=['gameId','playId'])

In [115]:
df_w2_on.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,off_xpass,def_xpass,pass_rate_off,pass_rate_def
0,2022091500,55,2,KC,LAC,20.0,8.0,9.0,3.0,8.0,1.0,10.0,6.0,39.0,0.680626,0.758184,0.625,0.763636


In [116]:
df_w2_on.week.min()

2

# Integrate week 1-imputed, week 2 onward data

where is 'week' lost?

In [117]:
w1_merged.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,pass_rate_def,pass_rate_off,off_xpass,def_xpass,Man_def,Zone_def,Cover-0_def,cover_1_def,Cover-2_def,cover_3_def,Quarters_def,cover_6_def,Other_def
0,2022090800,56,1,BUF,LA,0.615627,0.619126,0.619126,0.615627,7.117647,37.117647,0.705882,6.352941,0.647059,20.823529,7.647059,7.705882,0.352941


In [ ]:
df_w2_on = df_w2_on.dropna()

In [118]:
w1_merged = w1_merged[df_w2_on.columns]

In [121]:
merged_base = pd.concat([w1_merged,df_w2_on],axis=0)

In [122]:
merged_base.head(1)

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,off_xpass,def_xpass,pass_rate_off,pass_rate_def
0,2022090800,56,1,BUF,LA,20.823529,7.705882,6.352941,7.647059,0.647059,0.705882,7.117647,0.352941,37.117647,0.619126,0.615627,0.619126,0.615627


In [123]:
merged_base.isna().sum()

gameId              0
playId              0
week                0
possessionTeam      0
defensiveTeam       0
cover_3_def       552
cover_6_def       552
cover_1_def       552
Quarters_def      552
Cover-2_def       552
Cover-0_def       552
Man_def           552
Other_def         552
Zone_def          552
off_xpass         512
def_xpass         535
pass_rate_off     562
pass_rate_def     606
dtype: int64

In [124]:
df_w2_on.isna().sum()

gameId              0
playId              0
week                0
possessionTeam      0
defensiveTeam       0
cover_3_def       552
cover_6_def       552
cover_1_def       552
Quarters_def      552
Cover-2_def       552
Cover-0_def       552
Man_def           552
Other_def         552
Zone_def          552
off_xpass         512
def_xpass         535
pass_rate_off     562
pass_rate_def     606
dtype: int64

In [128]:
df_w2_on[df_w2_on['cover_3_def'].isna()]

,gameId,playId,week,possessionTeam,defensiveTeam,cover_3_def,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,off_xpass,def_xpass,pass_rate_off,pass_rate_def
3529,2022092513,202,3,DEN,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.521677,NaN,0.617886,NaN
3536,2022092513,466,3,DEN,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.521677,NaN,0.617886,NaN
3537,2022092513,487,3,DEN,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.521677,NaN,0.617886,NaN
3538,2022092513,532,3,DEN,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.530023,NaN,0.617886,NaN
3543,2022092513,715,3,DEN,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500497,NaN,0.617886,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12484,2022103012,3671,8,GB,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.766549,NaN,0.618483,NaN
12485,2022103012,3693,8,GB,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.762680,NaN,0.618483,NaN
12486,2022103012,3717,8,GB,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.762680,NaN,0.618483,NaN
12487,2022103012,3739,8,GB,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.743281,NaN,0.618483,NaN


In [129]:
cu_df.isna().sum()

defensiveTeam    0
week             0
cover_3_def      0
cover_6_def      0
cover_1_def      0
Quarters_def     0
Cover-2_def      0
Cover-0_def      0
Man_def          0
Other_def        0
Zone_def         0
dtype: int64